In [1]:
from binance.client import Client
import pandas as pd


api_key="mgMDvJNtetK1AukW4EOroL1Vq9dUIrkltYzBTATzXzCS9mX7a95nWIvvxoMAYcug"
api_secret="sBZlBhLd5G0wEVDX3v47lZuAAAsreY9PJDvPEZ1G7f11uCXPjb2likvAJwcbdeGb"

API_KEY = api_key
API_SECRET = api_secret

client = Client(api_key, api_secret)  # mainnet
symbol = "BTCUSDT"
interval = Client.KLINE_INTERVAL_1MINUTE
start_str = "1 Nov, 2025"

klines = client.get_historical_klines(symbol, interval, start_str)

df = pd.DataFrame(klines, columns=[
    "open_time","open","high","low","close","volume",
    "close_time","quote_asset_volume","number_of_trades",
    "taker_buy_base_asset_volume","taker_buy_quote_asset_volume","ignore"
])

df["open_time"] = pd.to_datetime(df["open_time"], unit='ms')
df["close_time"] = pd.to_datetime(df["close_time"], unit='ms')
for col in ["open","high","low","close","volume"]:
    df[col] = df[col].astype(float)

print(df.head())


            open_time       open       high        low      close    volume  \
0 2025-11-01 00:00:00  109608.01  109621.62  109608.00  109616.35   5.65926   
1 2025-11-01 00:01:00  109616.34  109639.64  109550.46  109550.47  47.19536   
2 2025-11-01 00:02:00  109550.46  109550.46  109470.92  109470.93  10.29554   
3 2025-11-01 00:03:00  109470.92  109476.12  109447.00  109447.00  39.15662   
4 2025-11-01 00:04:00  109447.00  109457.94  109441.26  109444.44   4.47815   

               close_time quote_asset_volume  number_of_trades  \
0 2025-11-01 00:00:59.999    620327.34965220              1005   
1 2025-11-01 00:01:59.999   5172298.30687730              2016   
2 2025-11-01 00:02:59.999   1127423.03182360              2208   
3 2025-11-01 00:03:59.999   4286187.85469350              2785   
4 2025-11-01 00:04:59.999    490126.77018130              1443   

  taker_buy_base_asset_volume taker_buy_quote_asset_volume ignore  
0                  3.81740000              418426.01560140  

In [2]:
import vectorbt as vbt
import numpy as np

# Make a vectorbt-compatible wrapper
def alpha_momentum_vbt(close, window, up_thresh, down_thresh):
    signals = []
    for i in range(len(close)):
        if i < window:
            signals.append(0)
        else:
            mean_window = close[i-window:i].mean()
            if close[i] > mean_window * up_thresh:
                signals.append(1)
            elif close[i] < mean_window * down_thresh:
                signals.append(-1)
            else:
                signals.append(0)
    return np.array(signals)

close = df['close']


edited manually, stop running 


In [8]:
param_grid = {
    'window': [5, 10, 15, 20,25,30,35,40,45,50,60,70,80],
    'up_thresh': [1.0005, 1.001, 1.002, 1.005,1.01,1.2,1.3,1.4,1.5],
    'down_thresh': [0.995, 0.997, 0.999, 0.9995,0.999,0.998,0.995,0.99]
}


In [5]:
df = df.set_index('open_time')
df = df.asfreq('1min')   # because Binance interval = 1 minute
close = df['close']


In [9]:
# Generate all combinations of parameters
from itertools import product

results = []

for w, up, down in product(param_grid['window'], param_grid['up_thresh'], param_grid['down_thresh']):
    signals = alpha_momentum_vbt(close, window=w, up_thresh=up, down_thresh=down)
    
    pf = vbt.Portfolio.from_signals(
    close,
    entries=signals == 1,
    exits=signals == -1,
    init_cash=10000,
    fees=0.001,
    freq='1min'
)

    
    # Save metrics
    results.append({
        'window': w,
        'up_thresh': up,
        'down_thresh': down,
        'total_return': pf.total_return(),
        'sharpe': pf.sharpe_ratio(),
        'max_drawdown': pf.max_drawdown()
    })

results_df = pd.DataFrame(results)
best_params = results_df.sort_values('sharpe', ascending=False).iloc[0]
print("Best parameters:\n", best_params)


C:\Users\vibhu\AppData\Local\Temp\ipykernel_932\3833069572.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if close[i] > mean_window * up_thresh:
C:\Users\vibhu\AppData\Local\Temp\ipykernel_932\3833069572.py:14: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  elif close[i] < mean_window * down_thresh:


Best parameters:
 window          80.00
up_thresh        1.50
down_thresh      0.99
total_return     0.00
sharpe            inf
max_drawdown     0.00
Name: 935, dtype: float64
